In [2]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI


from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI

from langchain_core.messages import HumanMessage

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver


from typing import List
from langchain_core.documents import Document
import os
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter


memory = MemorySaver()
config = {"configurable": {"user_id": "3", "thread_id": "1"}}

In [3]:
!ls

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [4]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [5]:


@tool
def multiply(a: int, b: int) -> int:
    """
    when you need to multiply use this tool
    """
    
    return a * b

@tool
def translate_to_rus(a: str) -> str:
    """
    every time when you return response put through this function first
    """

    return 'some_text'






In [6]:
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

тул, который делает запрос к бд

In [7]:
tools = [multiply,translate_to_rus]

In [8]:

model_with_tools = llm.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="solve four times 3 ")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [19]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)


In [20]:
response = agent_executor.invoke({"messages": [HumanMessage(content="solve four times 3")]})

response["messages"][-1].content

ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']

In [21]:
for step, metadata in agent_executor.stream(
        {"messages": [HumanMessage(content="my favourite color is red")]},
        config,
        stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")
        # print(text)

Мой любимый цвет - красный.

In [22]:
for step, metadata in agent_executor.stream(
        {"messages": [HumanMessage(content="which is my favourite colour?")]},
        config,
        stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")
        # print(text)

Your favourite color is red.

In [23]:
import pandas as pd

In [24]:
data = pd.read_csv('weather.csv')

In [30]:
data.head()

NameError: name 'data' is not defined

собрать документы, pdf. 
rag системы 

In [28]:
folder_path = "documents"
documents = load_documents(folder_path)

len(documents)

1

In [34]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 64 chunks.


In [9]:
!pwd

"pwd" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [10]:
#model_name='models/Linq-Embed-Mistral'
model_name='models/multilingual-e5-large-instruct'
embeddings = SentenceTransformerEmbeddings(model_name=model_name)

C:\Users\Peter\AppData\Local\Temp\ipykernel_20608\2979046391.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=model_name)


In [11]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=documents,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

NameError: name 'documents' is not defined

In [ ]:
query = "какие метрики качества используются?"
search_results = vectorstore.similarity_search(query, k=3)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1



Top 2 most relevant chunks for the query: 'какие метрики качества?'

Result 1:
Source: documents\Результаты анализа с ГенИИ (Русский вариант).docx
Content: Тема: «Оптимальные методы прогнозирования ликвидности в цветной металлургии».



Подготовлено BeringPro совместно с ГенИИ для Норникеля.




Оглавление

	1. Локальный vs. групповой подход к прогнозированию параметров	3

	2. Точные методы для локального прогнозирования различных параметров	4

	3. Прогнозирование групп взаимосвязанных параметров (групповой подход)	7

	4. Метрики оценки точности прогнозов	10

	5. Сценарное моделирование и стресс-тестирование	12

	6. Факторный анализ и чувствительность результатов прогноза	15

	7. Рекомендации по допустимой точности и материалальности	18

	8. Инструменты расчетов и визуализации (Python, «Форсайт» и альтернативы)	20

	9. Степень достоверности прогнозов по группам показателей	27

	10. Методы сбора и анализа исторической точности прогнозов	29

	11. Рекомендации по допустимым уровням ошибо